In [ ]:
import matplotlib.pyplot as plt
from sympy import symbols, Eq, solve


# Создание классов вершин и ребр графа. Скважины и сток будут особыми узлами с отдельными классами
class Well:
    def __init__(self, idx, alpha, const, Q):
        self.idx = idx
        self.alpha = alpha
        self.const = const
        self.Q = Q
        self.edge_output = []
        self.Q_vertex_list = [self.Q]

class Vertex:
    def __init__(self, idx):
        self.idx = idx
        self.edge_input = []
        self.edge_output = []
        self.Q = 0
        self.Q_vertex_list = []
        self.visited = False
        self.k = []
        self.count = 0
        
    def update_Q(self):
        self.Q_vertex_list = self.Q_vertex_list + self.edge_input[self.count].vertex1.Q_vertex_list
        self.Q = self.Q + self.edge_input[self.count].Q_edge
        self.count += 1
        self.visited = False

class Edge:
    def __init__(self, idx, vertex1, vertex2, length, diameter, roughness, density = 1000):
        self.idx = idx
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.double_index = str(self.vertex1.idx) + str(self.vertex2.idx)
        self.length = length
        self.diameter = diameter
        self.roughness = roughness
        self.density = density
        self.pot = self.diameter / (self.roughness * self.length) # Коэффициент распределения дебита при разветвлении на графе с привязкой характеристик труб
        self.Q_edge = 0

    def update_Q_edge(self, k = None):
        if len(self.vertex1.edge_output)>1:
            self.x = symbols('x')
            self.eqs = Eq(self.x * sum(edge.pot for edge in self.vertex1.edge_output), 1)
            self.sol_x = solve(self.eqs, self.x)
            self.sol_x = self.sol_x[0]
            self.prop_koef = self.sol_x * self.pot
            
            for i in range(len(self.vertex1.Q_vertex_list)):
                self.Q_edge = self.Q_edge + self.vertex1.Q_vertex_list[i] * k[i]

            self.Q_edge = self.Q_edge * self.prop_koef
            self.pressure_loss = self.density * self.length * self.Q_edge * self.roughness / self.diameter

        else:
            self.Q_edge = self.vertex1.Q
            self.pressure_loss = self.density * self.length * self.Q_edge * self.roughness / self.diameter

class Drain:
    def __init__(self, p_0, alpha, const, idx=0):
        self.p_0 = p_0
        self.edge_input = []
        self.alpha = alpha
        self.const = const
        self.idx = idx
        self.visited = False
        self.Q_0 = self.alpha * self.p_0 + self.const

# Создание графа
def create_graph(p_0, dw_parameters, edge_parameters, edge_vertexes):
    # alpha = 3
    # const = 10

    # Сток
    # dw_parameters = [] # Alpha и Const для стока и скважин
    # alpha = int(input("Enter alpha for Drain: "))
    # const = int(input("Enter const for Drain: "))
    # dw_parameters.append([alpha, const]) # Drain Well Alpha and Const

    drain = [Drain(p_0, dw_parameters[0][0], dw_parameters[0][1])]

    # Количество скважин
    # num_wells = int(input("Enter the number of Wells: "))
    num_wells = len(dw_parameters) - 1
    wells = []
    
    # Создание списка из num_wells элементов - Q1, Q2, Q3
    flow_rate = []
    flow_rate = symbols(['Q{}'.format(i+1) for i in range(num_wells)])
    
    counter = 0
    for i in range(num_wells):
        counter = counter + 1
        # alpha = int(input("Enter alpha for well {}: ".format(i + 1)))
        # const = int(input("Enter const for well {}: ".format(i + 1)))
        # dw_parameters.append([alpha, const])
        wells.append(Well(i+1, dw_parameters[i+1][0], dw_parameters[i+1][1], flow_rate[i]))

    # Количество дополнительных вершин
    num_vertex = int(input("Enter the number of Vertex: "))
    vertex = []
    for i in range(num_vertex):
        vertex.append(Vertex(i+1 + counter))

   
    common_vertex = drain + wells + vertex # A common array of all vertices
    print(common_vertex)

    # Количество ребр (труб)
    # num_edges = int(input("Enter the number of Edges: "))
    num_edges = len(edge_parameters)

    edges = []

    # edge_parameters = []
    # edge_vertexes = []

    for i in range(num_edges):
        # vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1)))
        # vertex2 = int(input("Enter the second vertex for edge {}: ".format(i + 1)))
        # length = float(input("Enter the length for edge {}: ".format(i + 1)))
        # diameter = float(input("Enter the diameter for edge {}: ".format(i + 1)))
        # roughness = float(input("Enter the roughness for edge {}: ".format(i + 1)))
        # edge_parameters.append([length, diameter, roughness])
        # edge_vertexes.append([vertex1, vertex2])
        edges.append(Edge(i+1, common_vertex[edge_vertexes[i][0]], common_vertex[edge_vertexes[i][1]], edge_parameters[i][0], edge_parameters[i][1], edge_parameters[i][2]))

        # edges.append(Edge(i+1, common_vertex[vertex1], common_vertex[vertex2]))

    for edge in edges:
        edge.vertex2.edge_input.append(edge)
        edge.vertex1.edge_output.append(edge)
        # print(edge.vertex1.edge_output)
        
    print(edges[-1].vertex1.edge_output)
    print(len(edges[0].vertex1.edge_output))
    print(len(edges[-1].vertex1.edge_output))


    counter_k = 0
    # for edge in edges:
    #     if len(edge.vertex1.edge_output) > 1:
    #         for i in range(len(edge.vertex1.Q_vertex_list)):
    #             counter_k = counter_k + 1
    #             k = k + symbols(['k{}'.format(counter_k)])
    # print(k)
        
    # Обновление характеристик дебитов на узлах и гранях, после создания всех элементов графа и их связей
    while edges[num_edges-1].Q_edge == 0:
        for edge in edges:
            if len(edge.vertex1.edge_output) > 1:
                print('edge '+ f'{edge.idx}' + ' Loop: ' + f'{edge.vertex1.Q_vertex_list}')
                print('edge '+ f'{edge.double_index}' + ' Loop: ' + f'{edge.vertex1.Q_vertex_list}')



                # for i in range(len(edge.vertex1.Q_vertex_list)):
                #     counter_k = counter_k + 1
                #     k = k + symbols(['k{}'.format(counter_k)])
                if edge.vertex1.k == []:
                    for i in range(len(edge.vertex1.Q_vertex_list)):
                        counter_k = counter_k + 1
                        edge.vertex1.k = edge.vertex1.k + symbols(['k{}'.format(counter_k)])


                edge.update_Q_edge(edge.vertex1.k)
                print(edge.Q_edge)


                if edge.vertex2.visited == False and edge.vertex2.idx != 0 :
                    edge.vertex2.update_Q()

            else:

                print('edge '+ f'{edge.idx}' + ' No Loop')
                edge.update_Q_edge()
                if edge.vertex2.visited == False and edge.vertex2.idx != 0 :
                    edge.vertex2.update_Q()
                # k.pop(0) - Неправильно

        # for vert in vertex:
        #     vert.update_Q()
    return wells, vertex, drain, edges, flow_rate, dw_parameters, edge_parameters, edge_vertexes, common_vertex

# Тестовые параметры для тестового графа (на 2 скважины, 3 узла, 5 ребр)
# Graph 1 
# dw_parameters = [[10, -50000], [20, -20e9], [30, -30e9]] # [[alpha, const]]
# edge_parameters = [[100.0, 2.0, 0.1], [120.0, 2.0, 0.1], [135.0, 2.0, 0.1], [96.0, 2.0, 0.1], [220.0, 5.0, 0.1]] #[[length, diametr, roughness]]
# edge_vertexes = [[1, 3], [2, 4], [3, 5], [4, 5], [5, 0]]

# Graph 2
# dw_parameters = [[100, 100], [10, 110], [40, -840], [32, -60], [16, 78]] # [[alpha, const]]
# edge_parameters = [[100.0, 2.0, 0.1], [80.0, 2.0, 0.1], [70.0, 2.0, 0.1], [85.0, 2.0, 0.1], [120.0, 5.0, 0.1], [60.0, 5.0, 0.1], [92.0, 5.0, 0.1], [320.0, 5.0, 0.1]] #[[length, diametr, roughness]]
# edge_vertexes = [[1, 5], [2, 5], [3, 6], [4, 7], [5, 8], [6, 8], [7, 8], [8, 0]]

# # Graph 3 with loop
# dw_parameters = [[10, -500], [10, -500], [10, -500]] # [[alpha, const]]
# edge_parameters = [[-11.0, 1.5, 0.1], [-10.0, 1.5, 0.1], [-8.0, 1.8, 0.1], [-9.0, 1.8, 0.1], [-16.0, 2.5, 0.1]] #[[length, diametr, roughness]]
# edge_vertexes = [[1, 3], [2, 3], [3, 4], [4, 0], [3, 0]]

# Original Graph
# dw_parameters = [[-10, 50], [-23, 300], [-60, 800], [-360, 500], [-200, 500], [-120, 500]] # [[alpha, const]] # Parameters of Wells + Drain
# dw_parameters = [[-10, 50000], [-23, 20e9], [-60, 30e9], [-36, 12e9], [-20, 6e7], [-12, 120000]] # [[alpha, const]] # Parameters of Wells + Drain
dw_parameters = [[10, -50000], [20, -20e9], [30, -30e9], [20, -20e9], [30, -30e9], [20, -20e9]] # [[alpha, const]] # Parameters of Wells + Drain


edge_parameters = [[11.0, 1.5, 0.1], [10.0, 1.5, 0.1], [8.0, 1.8, 0.1], [9.0, 1.8, 0.1], [16.0, 2.5, 0.1], [11.0, 1.5, 0.1], [10.0, 1.5, 0.1], [8.0, 1.8, 0.1], [9.0, 1.8, 0.1], [16.0, 2.5, 0.1]] #[[length, diametr, roughness]]
edge_vertexes = [[1, 6], [2, 6], [3, 7], [4, 8], [5, 8], [6, 7], [8, 7], [7, 9], [8, 9], [9, 0]] # Vertex Conections

p_0 = 100

# Создание графа
wells, vertex, drain, edges, flow_rate, dw_parameters, edge_parameters, edge_vertexes, common_vertex = create_graph(p_0, dw_parameters, edge_parameters, edge_vertexes)

In [ ]:
# Check Debits on Edges
for edge in edges:
    print('edge ' + f'{edge.double_index}: ' + f'{edge.Q_edge}')
    if len(edge.vertex1.edge_output) > 1:
        print(edge.prop_koef)

In [ ]:
# print(vertex[0].edge_input)
# Finding all graph edges leading from the well to the drain
# def find_pipes(well, edges, drain):
#     pipes = []
#     visited = set()
#     stack = []
#     stack.append((well, []))
#     while stack:
#         curr, path = stack.pop()
#         visited.add(curr)
#         for edge in edges:
#             if edge.vertex1 == curr and edge.vertex2 not in visited:
#                 new_path = path[:] + [edge]
#                 if edge.vertex2 == drain:
#                     pipes.append(new_path)
#                 else:
#                     stack.append((edge.vertex2, new_path))
#     return pipes


In [ ]:
def find_pipes(well, edges, drain):
    pipes = []
    stack = []
    stack.append((well, []))
    while stack:
        curr, path = stack.pop()
        for edge in edges:
            if edge.vertex1 == curr and edge.vertex2 not in path:  # Modify this line
                new_path = path[:] + [edge]
                if edge.vertex2 == drain:
                    pipes.append(new_path)
                else:
                    stack.append((edge.vertex2, new_path))
    return pipes


pipes_from_wells_to_drain = []
for well in wells:
    pipes_from_wells_to_drain.append(find_pipes(well, edges, drain[0]))

In [ ]:
print(pipes_from_wells_to_drain)
print(len(pipes_from_wells_to_drain))
for well in range(len(wells)):
    print(len(pipes_from_wells_to_drain[well]))
# Проверка путей:
for well in range(len(wells)):
    for way in range(len(pipes_from_wells_to_drain[well])):
        for edge in pipes_from_wells_to_drain[well][way]:
            print('edge: ' + f'{edge.double_index}')
        print('end way ------------')
    print('end well -----------')
# # Проверка дебитов на последних трубах
# print(pipes_from_wells_to_drain[0][0][-1].Q_edge)
# print(pipes_from_wells_to_drain[0][0][-1].idx)
# print(pipes_from_wells_to_drain[0][1][-1].Q_edge)
# print(pipes_from_wells_to_drain[0][1][-1].idx)

# for i in range(len(pipes_from_wells_to_drain[1][0])):
#     print(pipes_from_wells_to_drain[1][0][i].idx)
#     print(pipes_from_wells_to_drain[1][0][i].pressure_loss)

In [ ]:
# Параметры alpha и const задаются непосредственно на скважинах, а не трубах !
p0 = drain[0].p_0
# Finding equations from each well's flow rate
def find_Q_well(p0, pipes_from_wells_to_drain):

    Q_well = []
    for well in range(len(wells)):
        for way in range(len(pipes_from_wells_to_drain[well])):
            pipes = pipes_from_wells_to_drain[well][way]
            pressure_loss = sum(edge.pressure_loss for edge in pipes)
            Q_well.append(wells[well].alpha * (p0 + pressure_loss) + wells[well].const)

    return Q_well

k_list = []
for vert in vertex:
    if vert.k != [pipes_from_wells_to_drain]:
        k_list = k_list + vert.k



In [ ]:
# flow_rate = []
# for well in range(len(wells)):
#     for way in range(len(pipes_from_wells_to_drain[well])):
#         flow_rate = flow_rate + symbols(['Q{}'.format(well+1)])

print(flow_rate)
print(k_list)
Q_well = find_Q_well(p0, pipes_from_wells_to_drain)
print(Q_well)
print(flow_rate)

In [ ]:
# flow_rate = []
# for well in range(len(wells)):
#     for way in range(len(pipes_from_wells_to_drain[well])):
#         flow_rate = flow_rate + symbols(['Q{}'.format(well+1)])
        
print(flow_rate)

# for i in range(len(Q_well)):
#     Q_well[i] = Q_well[i]- flow_rate[i]


In [ ]:
from sympy import symbols, lambdify
import numpy as np
from scipy.optimize import least_squares
def find_Q_drain_2(drain, p0, pipes_from_wells_to_drain, flow_rate, k_list):

    # flow_rate_origin = symbols(['Q{}'.format(i+1) for i in range(len(wells))])
    residual_history = []

    def residual(vars, *params):
        p0 = drain[0].p_0
        Q_well = find_Q_well(p0, pipes_from_wells_to_drain)
        # print('Q_well: ' + f'{Q_well}')
        # print('len(Q_well): ' + f'{len(Q_well)}')

        # Создание списка переменных для Q_well (Отличается размерность с обычным)
        flow_rate_multi = []
        for well in range(len(wells)):
            for _ in range(len(pipes_from_wells_to_drain[well])):
                flow_rate_multi = flow_rate_multi + symbols(['Q{}'.format(well+1)])
        # print('flow_rate_multi: '+ f'{flow_rate_multi}')

        # Вычитание правой части уравнения из левой
        for i in range(len(Q_well)):
            Q_well[i] = Q_well[i]- flow_rate_multi[i]
        # print(Q_well)

        # flow_rate_origin = symbols(['Q{}'.format(i+1) for i in range(len(wells))])
        # k_list = [k for vert in vertex if vert.k != pipes_from_wells_to_drain for k in vert.k]
        # common_list_var = flow_rate_origin + k_list
        common_list_var = flow_rate + k_list
        # print('common_list_var: ' + f'{common_list_var}')
        # print('Q_well: ' + f'{Q_well}')
        
        r = np.zeros(len(Q_well))
        for i in range(len(Q_well)):
            # Convert the symbolic expression to a numerical function
            Q_well_num = lambdify(common_list_var, Q_well[i], "numpy")
            
            # Evaluate the numerical function with the current variable values
            r[i] = Q_well_num(*vars)
        

        residual_norm = np.linalg.norm(r)
        residual_history.append(residual_norm)
        # print('r: ' + f'{r}')
        # print('len(r): ' + f'{len(r)}')
        return r
    
    # initial_guess = np.ones(len(flow_rate_origin) + len(k_list))  # Initial guess_1 for Q1, Q2, k1, k2
    # initial_guess = np.zeros(len(flow_rate_origin) + len(k_list))  # Initial guess_2 for Q1, Q2, k1, k2
    # initial_guess = np.random.randint(1, 15, len(flow_rate_origin) + len(k_list)).tolist()  # Initial guess_3 for Q1, Q2, k1, k2

    initial_guess = np.random.randint(1, 15, len(flow_rate) + len(k_list)).tolist()  # Initial guess_3 for Q1, Q2, k1, k2
    # print('initial_guess: ' + f'{initial_guess}')
    # print('len(initial_guess): ' + f'{len(initial_guess)}')


    result = least_squares(residual, initial_guess, method = 'lm')
    solution = result.x

    # # Split the solution into Q and k values
    # Q_values = solution[:len(flow_rate_origin)]
    # k_values = solution[len(flow_rate_origin):]

    Q_values = solution[:len(flow_rate)]
    k_values = solution[len(flow_rate):]
    alpha = drain[0].alpha
    const = drain[0].const
    # Поиск Q0
    Q0 = sum(Q_values)
    # Finding a new p0
    p0 = (Q0-const)/alpha
    
    return Q_values, k_values, Q0, p0, residual_history


In [ ]:
# print(f'Q_values: {Q_values}')
# print(f'k_values: {k_values}')
# for i in range(len(pipes_from_wells_to_drain[1][1])):
#     print(pipes_from_wells_to_drain[1][1][i].idx)
#     print(pipes_from_wells_to_drain[1][1][i].Q_edge)
#     print(pipes_from_wells_to_drain[1][1][i].pressure_loss)
#     # print('Pressure Loss Well_1 Way_1: ', + str(pipes_from_wells_to_drain[1][1][i].pressure_loss))
#     # print('Pressure Loss Well_1 Way_2: ', + str(pipes_from_wells_to_drain[1][2][i].pressure_loss))
# print(vertex[1].Q)
# print(vertex[1].edge_input)
# print(vertex[1].edge_output)
# print(vertex[1].Q_vertex_list)
# def find_Q_drain(drain, p0, pipes_from_wells_to_drain, flow_rate, k_list):

#     alpha = drain[0].alpha
#     const = drain[0].const

#     Q_well = find_Q_well(p0, pipes_from_wells_to_drain)
#     print(Q_well)

#     flow_rate_origin = symbols(['Q{}'.format(i+1) for i in range(len(wells))])
#     common_list_var = flow_rate_origin + k_list

#     # Solving a system of equations from Q wells
#     eqs = [Eq(Q_well[i], flow_rate[i]) for i in range(len(Q_well))]
#     print(eqs)
#     print(common_list_var)
#     sol = solve(eqs, common_list_var)
#     print(sol)
#     # Finding a new Q0
#     Q0 = sum(sol.get(well) for well in flow_rate_origin)

#     # Finding a new p0
#     p0 = (Q0-const)/alpha
#     return Q0, p0, Q_well
# import numpy as np
# from scipy.optimize import least_squares

# def residual(vars, *params):
#     Q1, Q2, k1, k2 = vars
#     r = np.zeros(4)
#     r[0] = 7332 * Q1 + 2623 * Q1 * k1 + 2623 * Q2 * k2 + 500
#     r[1] = 7332 * Q1 + 5574 * Q1 * k1 + 5574 * Q2 * k2 + 500
#     r[2] = 6665 * Q1 + 2623 * Q1 * k1 + 2623 * Q2 * k2 + 500
#     r[3] = 6665 * Q1 + 5574 * Q1 * k1 + 5574 * Q2 * k2 + 500
#     print(Q1)
#     return r

# initial_guess = np.array([1, 1, 1, 1])  # Initial guess for Q1, Q2, k1, k2
# result = least_squares(residual, initial_guess)
# Q1, Q2, k1, k2 = result.x

# print("Q1:", Q1)
# print("Q2:", Q2)
# print("k1:", k1)
# print("k2:", k2)

In [ ]:
Q0_prev = drain[0].Q_0
eps = 0.001

iterations = []
Q0_values = []

# set the maximum number of iterations
max_iterations = 10

# Итерационный поиск
i = 0
while i < max_iterations:
    # Q0, p0, Q_well = find_Q_drain(drain, p0, pipes_from_wells_to_drain, flow_rate, k_list)
    Q_values, k_values, Q0, p0, residual_history = find_Q_drain_2(drain, p0, pipes_from_wells_to_drain, flow_rate, k_list)

    print(residual_history)
    plt.plot(residual_history)
    plt.xlabel('Iteration')
    plt.ylabel('Residual norm')
    plt.title('Convergence of the numerical solution')
    plt.grid(True)
    plt.show()

    iterations.append(i)
    Q0_values.append(Q0)
    if abs(Q0 - Q0_prev) < eps:
        break

    Q0_prev = Q0

    i += 1

print("Q0:", Q0)
print("p0:", p0)

plt.plot(iterations, Q0_values)
# plt.semilogy() 
plt.xlabel('Number of iterations')
plt.ylabel('Q0 value')
plt.title('Convergence of Q0 values')
plt.show()

# Plot the residual history
# plt.plot(residual_history)
# plt.xlabel("Iteration")
# plt.ylabel("Sum of squared residuals")
# plt.title("Convergence of the numerical method")
# plt.show()

# Нахождение целевой функции
def find_k(Q0, edges):
    k = Q0 - sum(edge.length * edge.diameter/edge.roughness for edge in edges)
    return k

k = find_k(Q0, edges)
print("k function:", k)